<u><h1>Exploring the Ebay-Kleinanzeigen Car Sales Dataset</h1></u>

<h2>Introduction</h2>
<p>Today, we are exploring a subset (50,000 data points) of the dataset (370,000+ data points) that is the German eBay's classifieds used cars section of the website. The original scrapped data set can be found <a href="https://www.kaggle.com/orgesleka/used-cars-database/data">here</a> and the objective will be cleaning it up and analysing any trends in the used car listings dataset.</p>

<h2>Data Dictionary</h2>
<p>The following are the columns/variables of the dataset along with some descriptions:</p>
<ul>
<li><b>dateCrawled</b> : when this ad was first crawled, all field-values are taken from this date</li>
<li><b>named</b> : "name" of the car</li>
<li><b>seller</b> : private or dealer</li>
<li><b>offerType</b></li>
<li><b>price</b> : the price on the ad to sell the car</li>
<li><b>abtest</b></li>
<li><b>vehicleType</b></li>
<li><b>yearOfRegistration</b> : at which year the car was first registered</li>
<li><b>gearbox</b></li>
<li><b>powerPS</b> : power of the car in PS</li>
<li><b>model</b></li>
<li><b>odometer</b> : how many kilometers the car has driven</li>
<li><b>monthOfRegistration</b> : at which month the car was first registered</li>
<li><b>fuelType</b></li>
<li><b>brand</b></li>
<li><b>notRepairedDamage</b> : if the car has a damage which is not repaired yet</li>
<li><b>dateCreated</b> : the date for which the ad at ebay was created</li>
<li><b>nrOfPictures</b> : number of pictures in the ad (unfortunately this field contains everywhere a 0 and is thus useless (bug in crawler!) )</li>
<li><b>postalCode</b> </li>
<li><b>lastSeenOnline</b> : when the crawler saw this ad last online</li>
</ul>
<h2>Importing the data</h2>

In [1]:
# importing the import libraries
import pandas as pd
import numpy as np

# read csv data into pandas dataframe
autos = pd.read_csv("autos.csv", encoding = "Latin-1")

<h2>Information about the data</h2>

In [2]:
# display the first 5 rows of the dataset
print(autos.head(5), "\n")

           dateCrawled                                               name  \
0  2016-03-26 17:47:46                   Peugeot_807_160_NAVTECH_ON_BOARD   
1  2016-04-04 13:38:56         BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik   
2  2016-03-26 18:57:24                         Volkswagen_Golf_1.6_United   
3  2016-03-12 16:58:10  Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...   
4  2016-04-01 14:38:50  Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...   

   seller offerType   price   abtest vehicleType  yearOfRegistration  \
0  privat   Angebot  $5,000  control         bus                2004   
1  privat   Angebot  $8,500  control   limousine                1997   
2  privat   Angebot  $8,990     test   limousine                2009   
3  privat   Angebot  $4,350  control  kleinwagen                2007   
4  privat   Angebot  $1,350     test       kombi                2003   

     gearbox  powerPS   model   odometer  monthOfRegistration fuelType  \
0    manuell      158  andere 

<b>Comment:</b> Based on the values in some of the colunms such as 'seller' and 'notRepaiedDamage', we see that the dataset is still in German and may have to be translated for easier analysis. Also, in the 'name' column, we see that some of the names are more than alphanumeric and includes some special accent characters (i.e. Ü) which may also have to be cleaned. Furthermore, the date time and numeric columns look okay except maybe the 'price' and 'odometer' column will have to be cleaned. This means the removal of the '$' sign and the comma in the 'price' column and the removal of the 'km' unit and the comma in the 'odometer' column.

In [3]:
# display info on the column variables
print(autos.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

<b>Comment:</b> Here, we see that most of the data is of type object (i.e. strings) and we have a few integer variables. But, we see that there is some object columns which are better off as floats or integers and these are the 'price' and 'odometer' columns . Furthermore, most of the columns have exactly 50000 data points, however as we see that there is several that have null data and we have either have to remove or assign values to these rows.

Also, the column names will have to be converted to Python's preferred snakecase instead of its current camelcase format (i.e. Camel Case: "dateCrawled" -> Snake Case: "data_crawled"). 

<h2>Data Wrangling</h2>

In [4]:
# print existing column names
print(autos.columns)

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')


<b>Comment:</b> So, lets rename the 'yearOfRegistration', 'monthOfRegistration', 'notRepairedDamage' and 'dateCreated' to more suitable names whilst still explaining the variables intended use. We can call them 'registration_year', 'registration_month', 'unrepaired_damage' and 'ad_created' respectively. Also, the rest of the column names can keep the same name and will also be converted to snakecase.

In [5]:
# import re package to use regex on the strings
import re

# function to clean the column names
def clean_column(col):
    col = re.sub('([A-Z])', r'_\1', col) # put a underscore before the capital letters
    col = col.lower() # convert all characters to lower case
    return col

# dictionary of the columns to rename
new_names = {'yearOfRegistration' : 'registration_year',
             'monthOfRegistration' : 'registration_month',
             'notRepairedDamage': 'unrepaired_damage',
             'dateCreated' : 'ad_created',
             'powerPS': 'power_ps'}

# create new list for the new column names
new_columns = []

# modify the columns to have more suitable variable names
for column in autos.columns:
    if (column in ["yearOfRegistration", "monthOfRegistration", "notRepairedDamage", "dateCreated","powerPS"]):
        column = new_names[column]       
    new_columns.append(clean_column(column))
    
# assign new columns as the datasets columns
autos.columns = new_columns
    
# display new cleansed columns
print(autos.columns)

Index(['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'nr_of_pictures', 'postal_code',
       'last_seen'],
      dtype='object')


<b>Comment:</b> Now, we can see that the new columns are all in snakecase, cleaned and still retains the intent of its use giving in the data dictionary. Also, we also decided to changed the 'powerPS' column seperately like the previous four columns we wanted to change, because the 'PS' part of it should be together, but out 'clean_column' function would of put underscores before each capital letter. Here, we have clear and understandable columns that we can use for our analysis which abides by the usual conventions of what we see in Python, such as the use of snakecase.

<h2>Data Exploration</h2>
<p>Now, let's explore the dataset deeper and focus on the actual data to see which areas need some cleaning up.</p>

In [6]:
# display descriptive statistics for all columns
print(autos.describe(include = "all"))

               date_crawled         name  seller offer_type  price abtest  \
count                 50000        50000   50000      50000  50000  50000   
unique                48213        38754       2          2   2357      2   
top     2016-03-22 09:51:06  Ford_Fiesta  privat    Angebot     $0   test   
freq                      3           78   49999      49999   1421  25756   
mean                    NaN          NaN     NaN        NaN    NaN    NaN   
std                     NaN          NaN     NaN        NaN    NaN    NaN   
min                     NaN          NaN     NaN        NaN    NaN    NaN   
25%                     NaN          NaN     NaN        NaN    NaN    NaN   
50%                     NaN          NaN     NaN        NaN    NaN    NaN   
75%                     NaN          NaN     NaN        NaN    NaN    NaN   
max                     NaN          NaN     NaN        NaN    NaN    NaN   

       vehicle_type  registration_year  gearbox      power_ps  model  \
cou

<b>Comment:</b> Here, we have a non informative column that is 'nr_of_pictures' as according to the data dictionary it was the number of photos in the ad but it seemed that this was not scrapped properly so all values are 0. This is evident above as we see the descriptive statistics for the column is all 0. Also, as we discussed before, we had to convert the 'price' and 'odometer' columns to numeric as they are currently objects or strings specifically. Furthermore, it seems that the columns 'seller' and 'offer_type' each have basically one re occuring value ('privat' and 'Angebot') in almost all of the rows (occurs 49999 times).

In [7]:
# remove the nr_of_pictures column
autos = autos.drop("nr_of_pictures", axis = 1)

# display the remaining columns
print(autos.columns)

Index(['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'postal_code', 'last_seen'],
      dtype='object')


In [8]:
# convert the 'price' column into numeric
price_col = autos["price"]
price_col = price_col.str.replace("[$,]","") # remove the $ and , from the price
autos["price"] = price_col.astype(int) # convert column to numeric and use new clean column
print(autos["price"].head(3)) # display the first 3 rows to show our conversion

0    5000
1    8500
2    8990
Name: price, dtype: int64


In [9]:
# convert the 'odometer' column into numeric
odometer = autos["odometer"]
odometer = odometer.str.replace("km", "") # remove the km at the end of each value
odometer = odometer.str.replace(",", "") # remove the comma
autos["odometer"] = odometer.astype(int) # replace the current column with the new numeric column of values
autos.rename({"odometer" : "odometer_km"}, axis = 1, inplace = True) # rename the column to a more suitable name
print(autos["odometer_km"].head(3)) # display the first 3 rows to show the final result

0    150000
1    150000
2     70000
Name: odometer_km, dtype: int64


Now that we have found some more problems within our dataset, we should explore further for disrepancies within our data and since we have converted the two above columns ('price' and 'odometer_km') to numeric, let's focus on analysing those newly converted columns.

In [10]:
# get the min and max values of the price column
min_price = autos["price"].min()
max_price = autos["price"].max()
print("Minimum Price ($):", min_price)
print("Maximum Price ($):", max_price)

Minimum Price ($): 0
Maximum Price ($): 99999999


<b>Comment:</b> It seems that we have a huge range between \$0 to \$99999999 of the listed used car prices, so we much check why the prices are so high that they are in the tens of millions range. I guess it is normal for prices to be $0 for used cars as some owners may like to disposed of their vehicles as they may have no value attached to them (i.e. not servicable or operational).

In [11]:
# display the indexes that have the max price
print(autos.loc[autos.loc[:,"price"] == 99999999, :].index.values)

[39705]


<b>Comment:</b> So, we see that the 39705th row contains this high priced car and we may assume that it was an error and since it is the only high value then it must be an outlier so we can remove the row entirely.

In [12]:
# display the top 10 car prices in the listings
print("Top 10 highest used car prices in the listings:")
print(autos["price"].nlargest(10))

Top 10 highest used car prices in the listings:
39705    99999999
42221    27322222
27371    12345678
39377    12345678
47598    12345678
2897     11111111
24384    11111111
11137    10000000
47634     3890000
7814      1300000
Name: price, dtype: int64


<b>Comment:</b> Now, after further checking of the top 10 prices of the used cars, we see that the next few prices are also a bit steep im my opinion. So, it would be wise to use a more considerable price range and the next reasonable price that looks good would be the price of \$3,890,000 (9th highest price) as anything above that maybe an accidental price being listed or the poster set the price a bit steep as to not sell the car at all. Thus, we should remove the other cars that are higher than our new max price ($3,890,000) and that should be our new dataset.

In [13]:
# remove rows that are higher than the new price range
min_range = 0
max_range = 3890000
autos = autos[autos["price"].between(min_range, max_range)]

# get the min and max values of the new price column
min_price = autos["price"].min()
max_price = autos["price"].max()
print("New Minimum Price ($):", min_price)
print("New Maximum Price ($):", max_price)

# check how many rows of data remaining
print(autos.shape)

New Minimum Price ($): 0
New Maximum Price ($): 3890000
(49992, 19)


<b>Comment:</b> Here, we now have a new maximum price of $3,890,000 and after getting the dimensions of our new dataframe, we see that there is 49992 rows of data left out of our initialy 50000 rows (i.e. 8 rows were deleted in this process).

In [14]:
# display descriptive statistics regarding the odometer column
print(autos["odometer_km"].describe())

count     49992.000000
mean     125734.017443
std       40041.246220
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64


<b>Comment:</b> In the odometer column, the descriptive statistics look ok for the values that it has (measures the kilometers driven by the car) and we see that the minimum and maximum range is reasonable (5000, 150000).

In [15]:
# display the frequency of each value in the odometer column
print(autos["odometer_km"].value_counts().sort_index())

5000        967
10000       264
20000       784
30000       789
40000       818
50000      1026
60000      1164
70000      1230
80000      1436
90000      1757
100000     2168
125000     5170
150000    32419
Name: odometer_km, dtype: int64


<b>Comment:</b> The frequency of the kilometers driven by the used cars seem to fall within a handful of values given above. Here, we see that a majority of the used cars has been driven for at least 150,000 kilometers (32419 cars) and it seems there is no non numeric value for any of the rows. So based on this, we can conclude that there is no further fixing required for the 'odometer_km' column.

<p>Now, that we have check the newly converted columns for any disrepancies and cleanse them, we should focus on the columns that contains dates to understand the times that these listings were made. Here, we have two categories of dates and they are either created by the web scraper to fetch these classifies data or dates that were from the actual page were the used car was listed (i.e. the seller's post date). From the data dictionary, we can find and seperate these columns into two categories, 'Crawler Dates' and 'Website Dates':</p>

Crawler Dates (Columns):
- 'date_crawled'
- 'last_seen'

Website Dates (Columns):
- 'ad_created'
- 'registration_month'
- 'registration_year'

Now, that we have all the date columns seperated, lets see what type of data they are.

In [16]:
# check the datatypes of the dataset
print(autos[['date_crawled', 'last_seen', 'ad_created', 'registration_month', 'registration_year']].dtypes)

date_crawled          object
last_seen             object
ad_created            object
registration_month     int64
registration_year      int64
dtype: object


<b>Comment:</b> So, there is 3 columns that are still object types and the other 2 are integer types, which are suitable for the column values they represent. Thus, we have to convert the first 3 columns into more suitable datatypes and we have examine a few rows to check their current format.

In [17]:
# display 3 rows of data regarding the columns date_crawled, last_seen and ad_created
print(autos[["date_crawled", "last_seen", "ad_created"]].head(3))

          date_crawled            last_seen           ad_created
0  2016-03-26 17:47:46  2016-04-06 06:45:54  2016-03-26 00:00:00
1  2016-04-04 13:38:56  2016-04-06 14:45:08  2016-04-04 00:00:00
2  2016-03-26 18:57:24  2016-04-06 20:15:37  2016-03-26 00:00:00


<b>Comment:</b> It seems that there is a consistent format that the 3 columns use and it is the datetime format in Python. The format starts with the date starting from the year down to the day  and then it is followed by the timestamp of the car listing.

We should focus on the website data more than the crawlers data as we want to analyze the eBay listings rather than the web scraper itself and that means we should get the frequency of the listings for each ad in terms of their post date.

In [18]:
# display the distribution of dates of when the ad was created
ad_dates = autos["ad_created"].str[:10].value_counts(normalize = True, dropna = False)
ad_dates = ad_dates.sort_index()
print(ad_dates)

2015-06-11    0.000020
2015-08-10    0.000020
2015-09-09    0.000020
2015-11-10    0.000020
2015-12-05    0.000020
2015-12-30    0.000020
2016-01-03    0.000020
2016-01-07    0.000020
2016-01-10    0.000040
2016-01-13    0.000020
2016-01-14    0.000020
2016-01-16    0.000020
2016-01-22    0.000020
2016-01-27    0.000060
2016-01-29    0.000020
2016-02-01    0.000020
2016-02-02    0.000040
2016-02-05    0.000040
2016-02-07    0.000020
2016-02-08    0.000020
2016-02-09    0.000040
2016-02-11    0.000020
2016-02-12    0.000060
2016-02-14    0.000040
2016-02-16    0.000020
2016-02-17    0.000020
2016-02-18    0.000040
2016-02-19    0.000060
2016-02-20    0.000040
2016-02-21    0.000060
                ...   
2016-03-09    0.033225
2016-03-10    0.031865
2016-03-11    0.032785
2016-03-12    0.036606
2016-03-13    0.016923
2016-03-14    0.035226
2016-03-15    0.033745
2016-03-16    0.030005
2016-03-17    0.031205
2016-03-18    0.013722
2016-03-19    0.033845
2016-03-20    0.037866
2016-03-21 

<b>Comment:</b> Here, we see that the highest percentage of listings for used cars were during the March and April months of 2016 as shown by the distribution above (second column given as %). 

Now, let's look at the other two columns that were also apart of the listings data, which were the 'registration_month' and 'registration_year'.

In [19]:
# display the descriptive statistics for registration_month
print("Registration Month:")
print(autos["registration_month"].describe())

# display the descriptive statistics for registration_year
print("\nRegistration Year:")
print(autos["registration_year"].describe())

Registration Month:
count    49992.000000
mean         5.723796
std          3.711938
min          0.000000
25%          3.000000
50%          6.000000
75%          9.000000
max         12.000000
Name: registration_month, dtype: float64

Registration Year:
count    49992.000000
mean      2005.074552
std        105.720930
min       1000.000000
25%       1999.000000
50%       2003.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64


<b>Comment:</b> For the 'registration_month' column we see the minimum month in the dataset is 0, which obviously is not possible, but this may just be because the month posted maybe omitted and thus the value 0 is assigned. However, for the 'registration_year' column we see that the minimum year is 1000 and the maximum year is 9999, which is also impossible and those rows will need removal as well.

Now, logic will imply that the registration year will surely be before the ad is posted, such that any registration year for a used car should not exceed the year that the ad is posted (in this dataset it is between 2015-2016). In this case, we must filter out rows of data that have the registration year for the car, exceeding the year the ad was posted, as this would be impossible in real life. Well, unless they can time travel then that would invoke more curiosity to those individuals that posted the ad!

In [20]:
# display the frequency of the registration years for the used cars
print(autos["registration_year"].value_counts().sort_index())

1000       1
1001       1
1111       1
1500       1
1800       2
1910       9
1927       1
1929       1
1931       1
1934       2
1937       4
1938       1
1939       1
1941       2
1943       1
1948       1
1950       3
1951       2
1952       1
1953       1
1954       2
1955       2
1956       5
1957       2
1958       4
1959       7
1960      33
1961       6
1962       4
1963       9
        ... 
2001    2702
2002    2533
2003    2727
2004    2737
2005    3015
2006    2708
2007    2304
2008    2231
2009    2098
2010    1597
2011    1634
2012    1323
2013     806
2014     665
2015     399
2016    1316
2017    1452
2018     491
2019       3
2800       1
4100       1
4500       1
4800       1
5000       4
5911       1
6200       1
8888       1
9000       2
9996       1
9999       4
Name: registration_year, Length: 97, dtype: int64


<b>Comment:</b> It, is clear there is some outlandish registration years for cars that make absolutely no sense. So, it would be wise to remove any years that exceed the year 2016 and any cars that were registered before 1910, to make the data more valid and to hide any time-travelling car enthusiast.

In [21]:
# filter out rows that exceed or go under certain registration years
min_rego_year = 1910
max_rego_year = 2016
autos = autos[autos["registration_year"].between(min_rego_year, max_rego_year)]

# recheck the registration range again
print(autos["registration_year"].value_counts(normalize = True).sort_index())

1910    0.000187
1927    0.000021
1929    0.000021
1931    0.000021
1934    0.000042
1937    0.000083
1938    0.000021
1939    0.000021
1941    0.000042
1943    0.000021
1948    0.000021
1950    0.000062
1951    0.000042
1952    0.000021
1953    0.000021
1954    0.000042
1955    0.000042
1956    0.000104
1957    0.000042
1958    0.000083
1959    0.000146
1960    0.000687
1961    0.000125
1962    0.000083
1963    0.000187
1964    0.000250
1965    0.000354
1966    0.000458
1967    0.000562
1968    0.000541
          ...   
1987    0.001562
1988    0.002957
1989    0.003769
1990    0.008225
1991    0.007413
1992    0.008142
1993    0.009267
1994    0.013744
1995    0.027321
1996    0.030070
1997    0.042231
1998    0.051081
1999    0.062451
2000    0.069843
2001    0.056266
2002    0.052747
2003    0.056786
2004    0.056995
2005    0.062784
2006    0.056391
2007    0.047978
2008    0.046458
2009    0.043688
2010    0.033256
2011    0.034026
2012    0.027550
2013    0.016784
2014    0.0138

<b>Comment:</b> We have now filtered out more rows of data and so the range of registration years on cars seem more reasonable. In terms of the distribution, it looks like that the registration years for cars are more skewed towards the modern era (1987-2016) and that may imply that the used cars listed for sale are within the past 30 years, in terms of their age.

Now, that we have cleaned a considerable amount of the dataset and filtered out the non sensical rows, let's look at the main attraction which is the brand of used cars. We can display the top 10 brand of used cars on sale and also the average price for each brand.

In [22]:
# fetch the brand of cars
brands = autos["brand"]

# display the frequency of the cars (top 10)
print("Top 10 car brands in the eBay listings and their frequencies:")
top_brands = brands.value_counts().nlargest(10)
print(top_brands)

# dictionary to store the aggregate of the top 10 car brand prices
brand_mean_prices = {}
for brand in top_brands.index:
    brand_df = autos.loc[autos.loc[:,"brand"] == brand, :]
    total_price = brand_df["price"].sum()
    mean_price = total_price/top_brands[brand]
    brand_mean_prices[brand] = mean_price

# display the mean sale prices of top 10 car brands
print("\nAverage prices for top 10 car brands:")
for brand in top_brands.index:
    template = "Car Brand: {brand} | Average Price: ${mean_price:0.2f}"
    print(template.format(brand = brand, mean_price = brand_mean_prices[brand]))
    

Top 10 car brands in the eBay listings and their frequencies:
volkswagen       10187
bmw               5284
opel              5194
mercedes_benz     4579
audi              4149
ford              3351
renault           2274
peugeot           1418
fiat              1242
seat               873
Name: brand, dtype: int64

Average prices for top 10 car brands:
Car Brand: volkswagen | Average Price: $5426.38
Car Brand: bmw | Average Price: $8334.65
Car Brand: opel | Average Price: $2876.72
Car Brand: mercedes_benz | Average Price: $8485.24
Car Brand: audi | Average Price: $9093.65
Car Brand: ford | Average Price: $3949.42
Car Brand: renault | Average Price: $2395.42
Car Brand: peugeot | Average Price: $3039.47
Car Brand: fiat | Average Price: $2711.80
Car Brand: seat | Average Price: $4296.49


<b>Comment:</b> So, it looks like that the average price of these top 10 car brands is under \$10000, with the lowest being Renault brand (\$2395.42) and the highest belonging to the Audi brand (\$9093.65).

Now, we should get the average mileage of these car brands and see if there is any connection between the average price and mileage.

In [27]:
# get the average mileage for each car brand
avg_mileage = {}
for brand in top_brands.index:
    brand_mileage = autos.loc[autos.loc[:,"brand"] == brand, :]
    brand_mileage = brand_mileage["odometer_km"].sum()
    average_mileage = brand_mileage/top_brands[brand]
    avg_mileage[brand] = average_mileage
    
# merge both average price and mileage into a single dataframe from the dictionaries
df = "$" + pd.Series(brand_mean_prices).round(2).astype(str)
df = pd.DataFrame(df, columns = ["average_price"])
df["average_mileage"] = pd.Series(avg_mileage).round(2).astype(str) + "km"
print(df)

              average_price average_mileage
audi               $9093.65     129287.78km
bmw                $8334.65     132434.71km
fiat                $2711.8     116553.95km
ford               $3949.42     124068.93km
mercedes_benz      $8485.24     130856.08km
opel               $2876.72     129223.14km
peugeot            $3039.47     127136.81km
renault            $2395.42     128183.82km
seat               $4296.49     121563.57km
volkswagen         $5426.38     128728.28km


<b>Comment:</b> Here, there does not seem to be a big correlation between the average price of brands of used cars and their mileage. As we can seem, the average mileage of all the top 10 car brads in this dataset, seem to exceed 100,000 km which means all the cars have high usage before they were put on sale. Also, regardless of the price, the distribution of the mileage seem to be about the same despite the big average price differences between the most expensive car brand (Audi) and least expensive (Renault).